## Installing Spektral i.e. Tensorflow 2 and Keras GNNs interface

In [61]:
!pip install spektral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importin the libraries

In [62]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
from spektral.layers import GCNConv, GlobalSumPool
import numpy as np

In [63]:
from spektral.datasets import TUDataset

dataset = TUDataset('PROTEINS', clean=True)

Successfully loaded PROTEINS.


## Applying a GCFFilter

In [64]:
from spektral.transforms import GCNFilter

dataset.apply(GCNFilter())

In [65]:
np.random.shuffle(dataset)
split = int(0.8 * len(dataset))
dataset_train, dataset_test = dataset[:split], dataset[split:]

<ipython-input-65-25b645aff65d>:1: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset)


## Subclassing the model of GNN

In [66]:
class Catheral(Model):
  def __init__(self, n_hidden, n_labels):
    super().__init__()
    self.graph_conv = GCNConv(n_hidden)
    self.pool = GlobalSumPool()
    self.dropout = Dropout(0.5)
    self.dense = Dense(n_labels, 'softmax')

  def call(self, inputs):
    out = self.graph_conv(inputs)
    out = self.dropout(out)
    out = self.dense(out)

    return out

## Training and Compiling the model

In [67]:
model = Catheral(32, dataset.n_labels)
model.compile('adam', 'categorical_crossentropy')

## Fitting the model

### creating a dataloader cause we can't collapse the dataset into one way

In [68]:
from spektral.data import BatchLoader

loader = BatchLoader(dataset_train, batch_size=32)

In [69]:
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=10)

Epoch 1/10


/usr/local/lib/python3.8/dist-packages/spektral/data/utils.py:221: UserWarning: you are shuffling a 'TUDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(a)


ValueError: ignored